<a href="https://colab.research.google.com/github/NULabTMN/ps1-PoojaKShanbhag/blob/master/PS1_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this experiment, you will explore the accuracy of sentiment classificaiton using different feature representations of text documents.

First, you will implement `createBasicFeatures`, which creates a sparse matrix representation of a collection of documents. For this exercise, you should have a feature for each word containing at least one alphabetic character. You may use the `numpy` and `sklearn` packages to help with implementing a sparse matrix.

Then, you will implement `createFancyFeatures`, which can specify at any other features you choose to help improve performance on the classification task.

The two code blocks at the end train and evaluate two models—logistic regression with L1 and L2 regularization—using your featurization functions. Besides held-out classification accuracy with 10-fold cross-validation, you will also see the features in each class given high weights by the model.

In [0]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut,KFold
import numpy as np

In [0]:
# read in the movie review corpus
def readReviews():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/cornell_reviews.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  return corpus

This is where you will implement two functions to featurize the data.

In [3]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct class for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

def createBasicFeatures(corpus):
    #Your code here
    text = []
    classes = []
    for doc in corpus:
      text.append(doc['text'])
      classes.append(doc['class'])
    oneHotVect = CountVectorizer(token_pattern='\w*[a-zA-Z]\w*')
    #print(oneHotVect.get_stop_words())
    texts = oneHotVect.fit_transform(text).toarray()
    vocab = oneHotVect.get_feature_names()
    # print("vocab size",len(vocab))
    return texts,classes,vocab

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createFancyFeatures(corpus):
  #Your code here
  text = []
  classes = []
  dictVocab = dict()
  i = 0
  for doc in corpus:
      classes.append(doc['class'])
      s= doc['text']
      st = s.lower()
      
      text.append(st)
      
      token = st.split()
      tokenNoPunc = str.maketrans('', '', string.punctuation)
      token = [w.translate(tokenNoPunc) for w in token]
      # remove tokens that are not alphabetic
      token = [word for word in token if word.isalpha()]
      
      stop_words = set(stopwords.words('english'))
    
      token = [w for w in token if not w in stop_words]
      
      token = [word for word in token if len(word) > 1]
      
      for w in token:
          # print(w)
          if w not in dictVocab.keys():
              dictVocab[w]= i
              i += 1

  oneHotVect = CountVectorizer(vocabulary = dictVocab)
  #print(oneHotVect.get_stop_words())
  texts = oneHotVect.fit_transform(text).toarray()
  vocab = oneHotVect.get_feature_names()
  return texts,classes,vocab

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class

def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=KFold(n_splits=10, shuffle=True, random_state=1))
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  class0_weight_sorted = model.coef_[0, :].argsort()
  class1_weight_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  class0_indicators = [vocab[i] for i in class0_weight_sorted[:termsToTake]]
  class1_indicators = [vocab[i] for i in class1_weight_sorted[:termsToTake]]

  if model.classes_[0] == "pos":
    return avg_score,class0_indicators,class1_indicators
  else:
    return avg_score,class1_indicators,class0_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)

In [0]:
corpus = readReviews()

Run the following to train and evaluate two models using basic features:

In [6]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.827000
The most informative terms for pos are: ['flaws', 'memorable', 'terrific', 'edge', 'sherri', 'excellent', 'perfectly', 'masterpiece', 'enjoyable', 'using', 'fun', 'overall', 'gas', 'solid', 'command', 'quite', 'follows', 'fantastic', 'different', 'liar']
The most informative terms for neg are: ['waste', 'mess', 'ridiculous', 'lame', 'worst', 'headed', 'awful', 'unfortunately', 'cheap', 'write', 'boring', 'superior', 'tedious', 'bad', 'jesse', 'terrible', 'poor', 'maybe', 'looks', 'jakob']
----------L2 Norm-----------
The model's average accuracy is 0.833000
The most informative terms for pos are: ['fun', 'great', 'back', 'quite', 'well', 'excellent', 'overall', 'seen', 'american', 'perfectly', 'yet', 'memorable', 'terrific', 'job', 'pulp', 'true', 'very', 'performances', 'solid', 'different']
The most informative terms for neg are: ['bad', 'unfortunately', 'worst', 'waste', 'nothing', 'only', 'script', 'boring', 'awf

In [7]:
X,y,vocab = createFancyFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.819500
The most informative terms for pos are: ['memorable', 'flaws', 'pace', 'terrific', 'overall', 'edge', 'excellent', 'master', 'others', 'follows', 'enjoyable', 'allows', 'hilarious', 'fantastic', 'different', 'gas', 'fun', 'grand', 'chemistry', 'filmmaking']
The most informative terms for neg are: ['waste', 'mess', 'ridiculous', 'headed', 'awful', 'worst', 'supposed', 'lame', 'tedious', 'unfortunately', 'cheap', 'write', 'designed', 'rent', 'weak', 'guess', 'reason', 'forward', 'looks', 'bad']
----------L2 Norm-----------
The model's average accuracy is 0.851500
The most informative terms for pos are: ['fun', 'great', 'well', 'overall', 'others', 'memorable', 'terrific', 'different', 'matrix', 'quite', 'pace', 'true', 'excellent', 'performances', 'perfectly', 'hilarious', 'entertaining', 'seen', 'job', 'especially']
The most informative terms for neg are: ['bad', 'worst', 'unfortunately', 'waste', 'supposed', 'reason'

Run the following to train and evaluate two models using extended features: